<br><br><b><font size=10> CONTINUOUS CONTROL</font></b>
#### <i>...implementation for Udacity Deep Reinforcement Learning 
<hr><hr>

## Initialize Imports for the notebook
This Notebook uses code from separate python files where most of the implementation is handled

In [1]:
import environment as E
from buffers import ReplayBuffer
from agent import D4PG_Agent
import models
from logger import Logger

import os.path
import time
import re
import sys
import importlib
import copy

import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T
# import multiprocessing as multi
# multi.cpu_count()

16

## Manually declare an ARGS class
<i> For testing code in the notebook, to take the place of argparser in the command line.

In [22]:
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        self.num_episodes = 1
        self.max_time = 50
        self.alr = 1e-4
        self.clr = 1e-4
        self.batch = 128
        self.buffer = 100000
        self.C = 4000        
        
args = Args()
def check_args():
    for arg in vars(args):
        if arg == "sep": continue
        print("{}: {}".format(arg.upper(), getattr(args, arg)))
        
        
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")        

<b>Confirm that the args are all set the way we want them.

In [3]:
check_args()

TRAIN: True
NOGRAPHICS: False
NUM_EPS: 10
ROLLOUT: 5
BATCHSIZE: 64
PRETRAIN: 1000
NUM_EPISODES: 1
MAX_TIME: 50


## Load the environment
<i> 
And print a bit of information contained in the wrapper class
    
Set the training mode to FALSE while interactively learning about the code

In [5]:
env = E.Environment(args, id=1)
print("State size:", env.state_size)
print("Action size:", env.action_size)
print("Num Agents:", env.agent_count)


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State size: 33
Action size: 4
Num Agents: 20


<hr>

# Test code as it's developed

<hr>

## Take random actions in the environment below 
<i>
-to check that code is working<br>
-to get familiar with the environment

In [7]:
env.train = False
env.reset()
max_steps = 100
scores = np.zeros(env.agent_count)
arewards = np.empty((max_steps, env.agent_count))
states = env.states
actions = np.zeros((20,4))
for i in range(max_steps):
    actions = np.random.randn(env.agent_count, env.action_size)
    #actions[:,3] += .01
    actions = np.clip(actions, -1, 1)
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    arewards[i] = rewards
    states = next_states
#     if np.any(dones):
#         break
#    i += 1
#print(arewards)
print(arewards.min())
print(arewards.max())
#print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))  


0.0
0.0


### Force-reload modules as they're updated
<i> This notebook was developed as the code is written in Atom, the below cell reloads the modules as they're needed.

In [35]:
import models
import agent
import buffers
importlib.reload(models)
importlib.reload(agent)
importlib.reload(buffers)
from agent import D4PG_Agent
env.train = True
env.reset()

### Load the AGENT!

In [36]:
d4pg_agent = D4PG_Agent(env.state_size, env.action_size, env.agent_count, args.alr, args.clr, args.batch, args.buffer, args.C)

d4pg_agent.initialize_memory(args.pretrain, env)

Initializing memory buffer.
Taking pretrain step 10... memory filled: 120/1000                      
Taking pretrain step 20... memory filled: 320/1000                      
Taking pretrain step 30... memory filled: 520/1000                      
Taking pretrain step 40... memory filled: 720/1000                      
Taking pretrain step 50... memory filled: 920/1000                      
Taking pretrain step 54... memory filled: 1000/1000                      
Done!


In [37]:
print(d4pg_agent.critic.fc1.weight.data)
print(d4pg_agent.critic_target.fc1.weight.data)

tensor([[ 8.5254e-04,  4.7867e-04, -2.5832e-03,  ..., -9.0814e-04,
          5.2277e-04, -6.8841e-04],
        [-2.9081e-03,  5.7473e-04, -1.5296e-03,  ..., -2.5960e-03,
          3.8295e-04, -4.7723e-04],
        [ 2.3757e-03, -1.5391e-03,  1.0018e-03,  ..., -1.3681e-03,
         -2.2576e-03,  2.4680e-07],
        ...,
        [ 1.3895e-03, -1.2050e-03,  2.0381e-03,  ..., -4.3795e-04,
          2.8565e-03,  2.7703e-03],
        [ 2.9882e-05,  1.6816e-03,  2.8541e-03,  ...,  1.1532e-03,
          1.5181e-03, -9.6197e-04],
        [-1.3362e-03, -1.0828e-03,  3.2171e-04,  ...,  8.2172e-04,
         -1.3715e-03,  1.6301e-03]], device='cuda:0')
tensor([[ 8.5254e-04,  4.7867e-04, -2.5832e-03,  ..., -9.0814e-04,
          5.2277e-04, -6.8841e-04],
        [-2.9081e-03,  5.7473e-04, -1.5296e-03,  ..., -2.5960e-03,
          3.8295e-04, -4.7723e-04],
        [ 2.3757e-03, -1.5391e-03,  1.0018e-03,  ..., -1.3681e-03,
         -2.2576e-03,  2.4680e-07],
        ...,
        [ 1.3895e-03, -1.2050

## Test out Actor actions without training
<i> Test the <b>Actor</b> network

In [31]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(30):
    actions = d4pg_agent.act(states)
    
    # Print sample actions returned by the ACTOR network
    print("ACTIONS:", actions[1])
    
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

ACTIONS: [-0.12098932 -0.40055102 -0.23867819  0.08929716]
ACTIONS: [-0.01125762  0.01922529  0.1803064  -0.13634835]
ACTIONS: [ 0.6851838  -0.63353723  0.7436383   0.27492934]
ACTIONS: [-0.4682395  -0.5609217  -0.74521905  0.36964744]
ACTIONS: [-0.14885904 -0.20841734  0.5564802   0.27468184]
ACTIONS: [-0.4888853   0.76849324 -0.1348718   0.6583199 ]
ACTIONS: [ 0.33386275  0.06558588  0.05500592 -0.27297986]
ACTIONS: [ 0.11389834  0.77768856 -0.10445781  0.23022468]
ACTIONS: [-0.07206534  0.23954432 -0.37595558 -0.1366579 ]
ACTIONS: [ 0.15717742 -0.03802539 -0.2943189   0.55542237]
ACTIONS: [ 0.22255601 -0.28300273  0.03865694 -0.1659834 ]
ACTIONS: [ 0.35342917  0.5857887  -0.08753023  0.5232274 ]
ACTIONS: [-0.6818558   0.1905649   0.19634193 -0.2011529 ]
ACTIONS: [0.05289635 0.20017947 0.36530805 0.08598615]
ACTIONS: [ 0.02903109 -0.50989187  0.55134094 -0.02935818]
ACTIONS: [-0.1647969  -0.45551774  0.7326171   0.21744782]
ACTIONS: [-0.6101171   0.02357598 -0.1282186   0.16894275]
A

## Test out Critic scores without training
<i> Test the <b>Critic</b> network

In [32]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(2):
    actions = d4pg_agent.act(states) 
    ns, rewards, dones = env.step(actions)
    scores += rewards
    
    # Print sample distributions returned by the CRITIC
    batch = d4pg_agent.memory.sample(batch_size=1)
    states = torch.cat(batch.state).to(DEVICE)
    actions = torch.cat(batch.action).float().to(DEVICE)
    rewards = torch.cat(batch.reward).to(DEVICE)
    next_states = torch.cat(batch.next_state).to(DEVICE) 

    dist, probs = d4pg_agent.critic_target(next_states, d4pg_agent.actor(next_states))
    proj_dist = d4pg_agent._get_targets(rewards, next_states)
    ldist, lprobs = d4pg_agent.critic(next_states, d4pg_agent.actor(next_states))
    
    print("DIST: ", dist)
    print("Local DIST: ", ldist)
#     print("PROBS: ", probs)
#     print("PROJECTED DIST: ", proj_dist)
    print("\n")
    
    states = ns
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

DIST:  tensor([[0.0149, 0.0166, 0.0122, 0.0199, 0.0156, 0.0281, 0.0148, 0.0267, 0.0185,
         0.0232, 0.0132, 0.0190, 0.0155, 0.0180, 0.0229, 0.0159, 0.0125, 0.0248,
         0.0230, 0.0221, 0.0251, 0.0157, 0.0151, 0.0314, 0.0177, 0.0172, 0.0197,
         0.0187, 0.0175, 0.0179, 0.0171, 0.0179, 0.0194, 0.0250, 0.0129, 0.0216,
         0.0217, 0.0263, 0.0148, 0.0218, 0.0229, 0.0179, 0.0200, 0.0259, 0.0173,
         0.0204, 0.0284, 0.0173, 0.0253, 0.0185, 0.0143]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Local DIST:  tensor([[0.0149, 0.0166, 0.0122, 0.0199, 0.0156, 0.0281, 0.0148, 0.0267, 0.0185,
         0.0232, 0.0132, 0.0190, 0.0155, 0.0180, 0.0229, 0.0159, 0.0125, 0.0248,
         0.0230, 0.0221, 0.0251, 0.0157, 0.0151, 0.0314, 0.0177, 0.0172, 0.0197,
         0.0187, 0.0175, 0.0179, 0.0171, 0.0179, 0.0194, 0.0250, 0.0129, 0.0216,
         0.0217, 0.0263, 0.0148, 0.0218, 0.0229, 0.0179, 0.0200, 0.0259, 0.0173,
         0.0204, 0.0284, 0.0173, 0.0253, 0.0185, 0.0143]], d

<hr><hr>

## Test out various Agent functionality

<hr><hr>

In [ ]:
args.alr = 1e-4
args.clr = 1e-4
args.batch = 128
args.buffer = 100000
args.C = 4000

In [ ]:
check_args()


In [ ]:
import models
import agent
import buffers
importlib.reload(models)
importlib.reload(agent)
importlib.reload(buffers)
from agent import D4PG_Agent
d4pg_agent = D4PG_Agent(env.state_size, env.action_size, env.agent_count, args.alr, args.clr, args.batch, args.buffer, args.C)
env.train = True
env.reset()
d4pg_agent.initialize_memory(args.pretrain, env)

In [ ]:
d4pg_agent.initialize_memory(args.pretrain, env)

In [ ]:
args.max_time = 10

In [ ]:
#w_old = torch.zeros(d4pg_agent.actor.output.weight.data[0].shape)
for episode in range(1, 2):
    # Begin each episode with a clean environment
    env.reset()
    # Get initial state
    states = env.states
    scores = np.zeros(env.agent_count)
    # Gather experience for a maximum amount of steps, or until Done,
    # whichever comes first
    for t in range(args.max_time):
        actions = d4pg_agent.act(states)
        next_states, rewards, dones = env.step(actions)
        d4pg_agent.step(states, actions, rewards, next_states)
        states = next_states

        scores += rewards
        if np.any(dones):
            break
    print("Episode rewards: ", scores.mean())
    #print(scores)
    #print(len(d4pg_agent.memory))
    d4pg_agent.reset_nstep_memory()

## Learn about how the Categorical Bellman step works

In [ ]:
actions = np.random.randn(env.agent_count, env.action_size)
actions = np.clip(actions, -1, 1).astype(np.float32)
states = torch.from_numpy(env.states).float()

In [ ]:
probs, log_probs = critic(states, torch.from_numpy(actions))

In [ ]:
print(probs.shape)
print(log_probs.shape)

In [ ]:
class Container():
    def __init__(self):
        pass
c = Container()
    

In [ ]:
vmin = -10
vmax = 10
natoms = 51
gamma = .99
atoms = torch.linspace(vmin, vmax, natoms)
delta_z = (vmax - vmin) / (natoms -1)
r = torch.tensor(rewards).unsqueeze(-1)

probs = probs.detach()
q_next = (probs * atoms).sum()


### projected atoms
<html><i>
<b>atoms.view(1,-1)</b> becomes shape [1, num_atoms]
<br>
<b>r</b> is unsqueezed in the last (-1) dimension, so it's shape [20,1]
<br>
the result is a tensor that holds an offset (projected) version of the atoms for each reward instance

In [ ]:
np.set_printoptions(suppress=False)

#### tz = projected atoms, atoms (values) projected by scaling and offsetting via the bellman equation


In [ ]:
tz = r + gamma * atoms.view(1,-1)
tz.clamp_(vmin, vmax)

#### computes "bj" from the pseudo-code

In [ ]:
b = (tz - vmin) / c.delta_z
b[0].numpy()

#### l/u in the psuedocode are LOWER and UPPER bounds on the supports

In [ ]:
l = b.floor().long()
u = b.ceil().long()
print(l[0])
print(u[0])

#### m_l/m_u are computed in the pseudocode under "distribute the probability of tz", but still a bit opaque to me on how it should be used

In [ ]:
ml = (u.float() + (l == u).float() - b) * c.probs
ml[0].numpy()

In [ ]:
mu = (b - l.float()) * c.probs
mu[0].numpy()

In [ ]:
target_prob = torch.tensor(np.zeros(probs.size()))
for i in range(target_prob.size(0)):
    target_prob[i].index_add_(0, l[i].long(), ml[i].double())
    target_prob[i].index_add_(0, u[i].long(), mu[i].double())
target_prob[0].numpy()

### Close the environment when finished with the code/training/etc

In [ ]:
env.close()